In [1]:
import tomllib
try:
    with open("../../config.toml", "rb") as f:
        toml_dict = tomllib.load(f)
except tomllib.TOMLDecodeError:
    print("Yep, definitely not valid.")

FileNotFoundError: [Errno 2] No such file or directory: '../../../config.toml'

In [5]:
from helper.utils import *

get_link_to_port()[4]


{1: [3, 3], 3: [2, 3]}